# Check out new version of this file: data_import.ipynb

In [1]:
import pandas as pd
import datetime

In [4]:
def prepare_row(raw_to_column):
    retRow = {}
    for key in raw_to_column.keys():
        retRow[raw_to_column[key]] = None
    return retRow

def read_file(path_to_file):
    """
        Function that reads .txt file on the given path and transforms it to pandas DataFrame. It's assumed that
        an empty line is a separator between entries in the file.
        Usage example:
            df = read_file("datasets/MatchedData/ratings_ba.txt")
            df.head()
    """
    
    f = open(path_to_file, "r", encoding = 'utf8')
    
    lines = f.readlines()
    
    raw_to_column = {"beer_name" : "Beer Name", "beer_id" : "Beer Id", "style" : "Style", "brewery_name" : "Brewery Name", "brewery_id" : "Brewery Id", "abv" : "Abv", "date" : "Date", "user_name" : "Username", "user_id" : "User Id", "appearance" : "Appearance", "palate" : "Palate", "taste" : "Taste", 'aroma' : 'Aroma', "overall" : "Overall", "rating" : "Rating", "text" : "Text", "review" : "Review"}
    
    #Preparing the dictionary for reading the next entry
    newRow = prepare_row(raw_to_column)
    
    dataFrameDict = {'Beer Name' : [], 'Beer Id' : [], 'Brewery Name' : [], 'Brewery Id' : [], 'Style' : [], 'Abv' : [], 'Date' : [], 'Username' : [], 'User Id' : [],'Appearance' : [], 'Aroma' : [],'Palate' : [],'Taste' : [],'Overall' : [],'Rating' : [],'Text' : [],'Review' : []}
    
    for line in lines:
        
        delim = line.find(":")
        
        if ':' not in line: #No delimiter found <=> empty line found <=> current entry has ended
            for key in newRow.keys():
                dataFrameDict[key].append(newRow[key])
            newRow = prepare_row(raw_to_column)
        else:
            key = line[:delim]
            val = line[delim+1:-1].strip()
            # Following lines are just converting the value to the corresponding data type
            if key == 'date':
                try:
                    newRow[raw_to_column[key]] = datetime.datetime.fromtimestamp(int(val))
                except:
                    newRow[raw_to_column[key]] = None
            else:
                if key == 'beer_id' or key == 'brewery_id':
                    try:
                        newRow[raw_to_column[key]] = int(val)
                    except:
                        newRow[raw_to_column[key]] = None
                else:
                    if key == 'abv' or key == 'appearance' or key =='aroma' or key == 'palate' or key == 'taste' or key == 'overall' or key =='rating':
                        try:
                            newRow[raw_to_column[key]] = float(val)
                        except:
                            newRow[raw_to_column[key]] = None
                    else:
                        if key == 'review':
                            try:
                                newRow[raw_to_column[key]] = bool(val)
                            except:
                                newRow[raw_to_column[key]] = None
                        else:
                            newRow[raw_to_column[key]] = val
                    
    #If file doesn't end with an empty line
    if len(newRow) != 0:
        for key in newRow.keys():
            dataFrameDict[key].append(newRow[key])
    
    df = pd.DataFrame(dataFrameDict)
    #Pandas will cast these values to float by default - because NaN's exist
    df['Beer Id'] = df['Beer Id'].astype(pd.Int64Dtype())
    df['Brewery Id'] = df['Brewery Id'].astype(pd.Int64Dtype())
    
    return df

In [5]:
df = read_file("datasets/BeerAdvocate/reviews.txt")

In [6]:
df.shape

(2589587, 17)

In [6]:
df.to_pickle('ba_ratings.pkl')

In [9]:
df2 = pd.read_pickle('ba_ratings.pkl')

In [11]:
df3 = read_file("datasets/BeerAdvocate/reviews.txt")

In [13]:
df3.to_pickle('ba_reviews.pkl')

In [85]:
import tarfile
import os

In [86]:
DATA_FILE = './datasets'

In [87]:
def extract_tar_or_gz(filepath, to_folder='.'):
    if filepath.endswith(".txt.gz"):
        gunzip_shutil(filepath, os.path.splitext(filepath)[0])
        os.remove(filepath)
        
    elif filepath.endswith("tar.gz"):
        gz_file = tarfile.open(filepath, "r:gz")
        gz_file.extractall(path=to_folder)
        gz_file.close()
    
    elif filepath.endswith(".tar"):
        tar_file = tarfile.open(filepath, "r:")
        tar_file.extractall(path=to_folder)
        tar_file.close()

In [91]:
def extract_file(filepath, to_folder):
    extract_tar_or_gz(filepath, to_folder)
    print(filepath)
    print(to_folder)
    
    directory = os.path.join(to_folder)
    for filename in os.listdir(directory):
        file = os.path.join(directory, filename)
        if os.path.isfile(file):
            print(file)
            extract_tar_or_gz(file)

In [92]:
import gzip
import shutil
def gunzip_shutil(source_filepath, dest_filepath, block_size=65536):
    with gzip.open(source_filepath, 'rb') as s_file, \
            open(dest_filepath, 'wb') as d_file:
        shutil.copyfileobj(s_file, d_file, block_size)

In [93]:
extract_file(DATA_FILE + '/test/BeerAdvocate.tar.gz', DATA_FILE + '/test2')

./datasets/test/BeerAdvocate.tar.gz
./datasets/test2
./datasets/test2\beers.csv
./datasets/test2\breweries.csv
./datasets/test2\ratings.txt
./datasets/test2\ratings.txt.gz
./datasets/test2\reviews.txt
./datasets/test2\reviews.txt.gz
./datasets/test2\users.csv


In [83]:
extract_file(DATA_FILE + '/RateBeer.tar.gz', DATA_FILE + '/test3')

./datasets/RateBeer.tar.gz
./datasets/test3
./datasets/test3/beers.csv
./datasets/test3/breweries.csv
./datasets/test3/ratings.txt.gz
./datasets/test3/reviews.txt.gz
./datasets/test3/users.csv


In [84]:
extract_file(DATA_FILE + '/matched_beer_data.tar.gz', DATA_FILE + '/test4')

./datasets/matched_beer_data.tar.gz
./datasets/test4
./datasets/test4/beers.csv
./datasets/test4/breweries.csv
./datasets/test4/ratings.csv
./datasets/test4/ratings_ba.txt.gz
./datasets/test4/ratings_rb.txt.gz
./datasets/test4/ratings_with_text_ba.txt.gz
./datasets/test4/ratings_with_text_rb.txt.gz
./datasets/test4/users.csv
./datasets/test4/users_approx.csv


In [98]:
def create_pickle_from_file(filepath):
    df = read_file(filepath)
    df.to_pickle(os.path.splitext(filepath)[0]+'.pkl')

In [97]:
print(os.path.splitext('./datasets/test2/reviews.txt')[0]+'.pkl')

./datasets/test2/reviews.pkl


In [99]:
create_pickle_from_file(DATA_FILE + '/test2/reviews.txt')

In [100]:
import pandas as pd

In [101]:
dff = pd.read_pickle(DATA_FILE + '/test2/reviews.pkl')

In [102]:
dff.head()

,Beer Name,Beer Id,Brewery Name,Brewery Id,Style,Abv,Date,Username,User Id,Appearance,Aroma,Palate,Taste,Overall,Rating,Text,Review
0,Régab,142544,Societe des Brasseries du Gabon (SOBRAGA),37262,Euro Pale Lager,4.5,2015-08-20 12:00:00,nmann08,nmann08.184925,3.25,2.75,3.25,2.75,3.0,2.88,"From a bottle, pours a piss yellow color with ...",None
1,Barelegs Brew,19590,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.5,2009-02-20 12:00:00,StJamesGate,stjamesgate.163714,3.00,3.50,3.50,4.00,3.5,3.67,Pours pale copper with a thin head that quickl...,None
2,Barelegs Brew,19590,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.5,2006-03-13 12:00:00,mdagnew,mdagnew.19527,4.00,3.50,3.50,4.00,3.5,3.73,"500ml Bottle bought from The Vintage, Antrim.....",None
3,Barelegs Brew,19590,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.5,2004-12-01 12:00:00,helloloser12345,helloloser12345.10867,4.00,3.50,4.00,4.00,4.5,3.98,Serving: 500ml brown bottlePour: Good head wit...,None
4,Barelegs Brew,19590,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.5,2004-08-30 12:00:00,cypressbob,cypressbob.3708,4.00,4.00,4.00,4.00,4.0,4.00,"500ml bottlePours with a light, slightly hazy ...",None
